In [1]:
import pandas as pd
import torch
from torch import nn
from torchsummaryX import summary

import time
from pathlib import Path

from ml.config import follower_mk1_config, follower_mk2_config, follower_mk3_config
from ml.train import find_optimize_price, get_lr
from ml.model import FollowerReactionModel, PriceOptimizerModel

In [2]:
def extract_data(excel_file, sheet_name, config):
    x, y = pd.read_excel(excel_file, sheet_name=sheet_name)[["Leader's Price", "Follower's Price"]].to_numpy().T
    if config['use_date']:
        x = [[data, date] for date, data in enumerate(x, 1)]
    else:
        x = [[data] for data in x]
    y = [[data] for data in y]
    return torch.tensor(x, device=config['device']), torch.tensor(y, device=config['device'])


In [3]:
def model_summary(model, config):
    inp = torch.rand(2 if config['use_date'] else 1)
    summary(model, inp)

In [4]:
def train(config, model, x, y):

    lr = config['train_follower_reaction_lr']
    device = config['device']
    no_change_epochs = config['train_follower_reaction_no_change_epochs']
    print_interval = config['train_follower_reaction_print_interval']
    round_digits = config['train_follower_reaction_round_digits']
    scheduler_patience = config['train_follower_reaction_scheduler_patience']
    output_dir = config['output_dir']

    loss_func = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=scheduler_patience)

    losses = []
    i = 1
    last_output_time = time.time()
    while len(losses) < no_change_epochs or len(set(losses[-no_change_epochs:])) > 1:
        epoch_losses = []
        for inp, tar in zip(x, y):
            model.train().to(device)
            inp = inp.to(device).float()
            tar = tar.to(device).float()
            out = model(inp)
            loss = loss_func(out, tar)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())

        loss = round(sum(epoch_losses) / len(epoch_losses), round_digits)
        scheduler.step(loss)
        losses.append(loss)
        curr_time = time.time()
        if print_interval and (curr_time - last_output_time >= print_interval):
            print(f'[train] [epochs={i}] [lr={get_lr(optimizer)}] loss: {loss:.4f}')
            last_output_time = curr_time
        i += 1

    model_dir = f'{output_dir}/model_state_dict.torch'
    print(model_dir)
    torch.save(model.state_dict(), model_dir)

In [5]:
from pprint import pprint

def experiment(config):
    pprint(config)
    excel_file = config['excel_file']
    sheet_name = config['sheet_name']
    device = config['device']
    init_price = config['find_optimal_price_init_price']
    output_dir = config['output_dir']

    Path(output_dir).mkdir(parents=True, exist_ok=True)

    x, y = extract_data(excel_file, sheet_name, config)

    reaction_model = FollowerReactionModel(config)
    model_summary(reaction_model, config)
    train(config, reaction_model, x, y)

    price_optimizer = PriceOptimizerModel(reaction_model,
                                          init_price=init_price,
                                          device=device,
                                          date=config['date'])
    price = find_optimize_price(config, price_optimizer)

    print(f'optimal price: {price.item()}')

In [6]:
# experiment(follower_mk1_config)

In [7]:
# experiment(follower_mk2_config)

In [11]:
# config = follower_mk2_config
# config.update({
#     'find_optimal_price_print_interval': None
# })
# follower_reaction_model = FollowerReactionModel(config)
# state_dict_dir = f'{config["output_dir"]}/model_state_dict.torch'
# follower_reaction_model.load_state_dict(torch.load(state_dict_dir))
# for date in range(101, 111):
#     price_optimizer = PriceOptimizerModel(follower_reaction_model,
#                                           init_price=config['find_optimal_price_init_price'],
#                                           device=config['device'],
#                                           date=date)
#     price = find_optimize_price(config, price_optimizer)
#     print(f'date={price.item()}')

follower_mk3_config.update({
    'output_dir': 'mk3_test',
    'use_date': True,
    'date': 101,
    'layer_config': [8, 'relu', 6, 'relu', 2, 1],
    'emb': True
})
experiment(follower_mk3_config)

{'date': 101,
 'device': 'cpu',
 'emb': True,
 'excel_file': '../data14.xls',
 'find_optimal_price_init_price': 1.7,
 'find_optimal_price_lr': 0.001,
 'find_optimal_price_no_change_epochs': 1000,
 'find_optimal_price_print_interval': 2,
 'find_optimal_price_round_digits': 4,
 'find_optimal_price_scheduler_patience': 50,
 'layer_config': [8, 'relu', 6, 'relu', 2, 1],
 'output_dir': 'mk3_test',
 'sheet_name': 'Follower_Mk3',
 'train_follower_reaction_lr': 0.01,
 'train_follower_reaction_no_change_epochs': 50,
 'train_follower_reaction_print_interval': 2,
 'train_follower_reaction_round_digits': 5,
 'train_follower_reaction_scheduler_patience': 10,
 'use_date': True}
                  Kernel Shape Output Shape Params Mult-Adds
Layer                                                       
0_emb                   [1, 2]          [2]    4.0       2.0
1_emb_linear            [2, 1]          [1]    3.0       2.0
2_layers.Linear_0       [2, 8]          [8]   24.0      16.0
3_layers.ReLU_1       